# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.59,81,78,8.57,PN,1732659235
1,1,silver city,32.7701,-108.2803,18.15,33,0,10.29,US,1732659236
2,2,colchani,-20.3000,-66.9333,20.05,24,84,3.16,BO,1732659237
3,3,port-aux-francais,-49.3500,70.2167,3.30,73,79,14.18,TF,1732659238
4,4,arauco,-37.2463,-73.3175,15.68,56,10,4.17,CL,1732658965


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df['Humidity'].astype(int)

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng','Lat',geo=True,tiles='OSM',size='Humidity',color='City')

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 36) & (city_data_df["Max Temp"] > 28)] #82.5F to 96.8F
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 25)]  #Mostly clear skies

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,acapulco de juarez,16.8634,-99.8901,28.90,65,0,5.14,MX,1732659215
44,44,kapa'a,22.0752,-159.3190,28.04,69,0,5.14,US,1732659287
58,58,farasan,16.7022,42.1183,28.17,73,0,3.05,SA,1732659304
120,120,kailua-kona,19.6406,-155.9956,29.20,67,20,7.72,US,1732659380
122,122,bubaque,11.2833,-15.8333,28.28,67,0,5.51,GW,1732659382
255,255,lazaro cardenas,17.9583,-102.2000,28.03,65,0,3.89,MX,1732659285
360,360,remire-montjoly,4.9167,-52.2667,28.02,74,0,3.60,GF,1732659673
384,384,port moresby,-9.4431,147.1797,28.68,74,20,1.54,PG,1732659702
424,424,acopiara,-6.0953,-39.4525,29.44,26,19,2.09,BR,1732659752
463,463,kuala terengganu,5.3302,103.1408,28.97,79,20,3.60,MY,1732659799


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
17,acapulco de juarez,MX,16.8634,-99.8901,65,
44,kapa'a,US,22.0752,-159.3190,69,
58,farasan,SA,16.7022,42.1183,73,
120,kailua-kona,US,19.6406,-155.9956,67,
122,bubaque,GW,11.2833,-15.8333,67,
255,lazaro cardenas,MX,17.9583,-102.2000,65,
360,remire-montjoly,GF,4.9167,-52.2667,74,
384,port moresby,PG,-9.4431,147.1797,74,
424,acopiara,BR,-6.0953,-39.4525,26,
463,kuala terengganu,MY,5.3302,103.1408,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accomodation.hotel',
    'apikey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    nearest_hotel = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    nearest_hotel = nearest_hotel.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = nearest_hotel["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
acapulco de juarez - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
farasan - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
port moresby - nearest hotel: No hotel found
acopiara - nearest hotel: No hotel found
kuala terengganu - nearest hotel: No hotel found
chokwe - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
17,acapulco de juarez,MX,16.8634,-99.8901,65,No hotel found
44,kapa'a,US,22.0752,-159.3190,69,No hotel found
58,farasan,SA,16.7022,42.1183,73,No hotel found
120,kailua-kona,US,19.6406,-155.9956,67,No hotel found
122,bubaque,GW,11.2833,-15.8333,67,No hotel found
255,lazaro cardenas,MX,17.9583,-102.2000,65,No hotel found
360,remire-montjoly,GF,4.9167,-52.2667,74,No hotel found
384,port moresby,PG,-9.4431,147.1797,74,No hotel found
424,acopiara,BR,-6.0953,-39.4525,26,No hotel found
463,kuala terengganu,MY,5.3302,103.1408,79,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng','Lat',geo=True,tiles='OSM',size='Humidity',color='City',hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)